In [1]:
%autosave 0
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Téléchargez la liste de stop words en français
nltk.download('stopwords')
nltk.download('punkt')

# Importez la liste de stop words en français
from nltk.corpus import stopwords
stop_words_french = set(stopwords.words('french'))

Autosave disabled


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df_test = pd.read_csv("Data/df_featEngin.csv")
df_test.head(10)

,index,Nombre_avis_publie,Note,Titres,Commentaire,Verifications,longCommentaire,longTitres,nb_Mots_Commentaire,nb_Mots_Titres,...,Titres_3grame_livraison_trop_longue,Titres_3grame_delai_livraison_trop,Titres_3grame_rapport_qualite_prix,Titres_3grame_livraison_rapide_conforme,Titres_3grame_livraison_peu_longue,Titres_3grame_livraison_rapide_prix,TextBlob_polarity_Commentaire,TextBlob_polarity_Titres,Saison_experience,Sentiment
0,0,1,5,Je commande mes pneus de motos et de voitures,Je commande mes pneus de motos et de voitures ...,1,215,45,31,9,...,0,0,0,0,0,0,0.271000,0.00,3,1
1,1,6,5,"Ne cherchez plus vos pneus, Allopneus le peut","BonjourLe site est genial, tres bien explique ...",1,315,45,48,8,...,0,0,0,0,0,0,0.400000,0.15,3,1
2,2,3,5,Tout etait parfait,"Tout etait parfait, timing parfait dans les da...",1,227,18,36,3,...,0,0,0,0,0,0,0.391667,0.51,3,1
3,3,4,5,Pneus d'un tres bon rapport qualite_,Pneus d'un tres bon rapport qualite prix. Les ...,1,157,36,22,6,...,0,0,0,0,0,0,0.607500,1.00,3,1
4,4,1,4,"Bon suivi de la commande,les questions_","Bon suivi de la commande,les questions sont im...",1,218,39,34,6,...,0,0,0,0,0,0,0.133333,0.70,3,1
5,5,1,5,"Excellent service, bien plus avantageux compar...",Ce service sur place est d un grand secours. R...,1,276,62,48,8,...,0,0,0,0,0,0,0.202083,0.21,3,1
6,6,1,5,commande jusqu'au montage,La commande sur internet c'est tres bien passe...,1,439,25,71,3,...,0,0,0,0,0,0,0.216923,0.00,3,1
7,7,4,5,Excellent,"RAS, le rapport qualite prix est tres correct....",1,217,9,39,1,...,0,0,0,0,0,0,0.367143,0.50,3,1
8,8,3,5,Fidele depuis de nombreuses annees j ai_,Fidele depuis de nombreuses annees j ai toujou...,1,112,40,19,7,...,0,0,0,0,0,0,0.282500,0.05,3,1
9,9,1,5,Meilleur rapport qualite prix,M'a ete recommande. Meilleur rapport qualite p...,1,183,29,30,4,...,0,0,1,0,0,0,0.207500,0.53,3,1


In [4]:
# (a) Supprimer les caractères spéciaux, chiffres, valeurs manquantes et mettre le texte en minuscules.
df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()
df_test['Commentaire'] = df_test['Commentaire'].fillna('') 
df_test['Titres'] = df_test['Titres'].str.replace('[^a-zA-Z\s]', '').str.lower()
df_test['Titres'] = df_test['Titres'].fillna('') 
#df_test = df_test.drop(columns=[])
df_test.head(10)

C:\Users\joan\AppData\Local\Temp\ipykernel_1896\2952274688.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()
C:\Users\joan\AppData\Local\Temp\ipykernel_1896\2952274688.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Titres'] = df_test['Titres'].str.replace('[^a-zA-Z\s]', '').str.lower()


,index,Nombre_avis_publie,Note,Titres,Commentaire,Verifications,longCommentaire,longTitres,nb_Mots_Commentaire,nb_Mots_Titres,...,Titres_3grame_livraison_trop_longue,Titres_3grame_delai_livraison_trop,Titres_3grame_rapport_qualite_prix,Titres_3grame_livraison_rapide_conforme,Titres_3grame_livraison_peu_longue,Titres_3grame_livraison_rapide_prix,TextBlob_polarity_Commentaire,TextBlob_polarity_Titres,Saison_experience,Sentiment
0,0,1,5,je commande mes pneus de motos et de voitures,je commande mes pneus de motos et de voitures ...,1,215,45,31,9,...,0,0,0,0,0,0,0.271000,0.00,3,1
1,1,6,5,ne cherchez plus vos pneus allopneus le peut,bonjourle site est genial tres bien explique p...,1,315,45,48,8,...,0,0,0,0,0,0,0.400000,0.15,3,1
2,2,3,5,tout etait parfait,tout etait parfait timing parfait dans les dat...,1,227,18,36,3,...,0,0,0,0,0,0,0.391667,0.51,3,1
3,3,4,5,pneus dun tres bon rapport qualite,pneus dun tres bon rapport qualite prix les me...,1,157,36,22,6,...,0,0,0,0,0,0,0.607500,1.00,3,1
4,4,1,4,bon suivi de la commandeles questions,bon suivi de la commandeles questions sont imm...,1,218,39,34,6,...,0,0,0,0,0,0,0.133333,0.70,3,1
5,5,1,5,excellent service bien plus avantageux compare...,ce service sur place est d un grand secours ri...,1,276,62,48,8,...,0,0,0,0,0,0,0.202083,0.21,3,1
6,6,1,5,commande jusquau montage,la commande sur internet cest tres bien passee...,1,439,25,71,3,...,0,0,0,0,0,0,0.216923,0.00,3,1
7,7,4,5,excellent,ras le rapport qualite prix est tres correct t...,1,217,9,39,1,...,0,0,0,0,0,0,0.367143,0.50,3,1
8,8,3,5,fidele depuis de nombreuses annees j ai,fidele depuis de nombreuses annees j ai toujou...,1,112,40,19,7,...,0,0,0,0,0,0,0.282500,0.05,3,1
9,9,1,5,meilleur rapport qualite prix,ma ete recommande meilleur rapport qualite pri...,1,183,29,30,4,...,0,0,1,0,0,0,0.207500,0.53,3,1


In [5]:
from gensim.models import FastText

# Replace 'path/to/pretrained/embeddings' with the actual path to your downloaded embeddings file
model = FastText.load_fasttext_format("FastText/cc.fr.300.bin.gz")

C:\Users\joan\AppData\Local\Temp\ipykernel_1896\2105882537.py:4: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format("FastText/cc.fr.300.bin.gz")


In [6]:
# Define preprocessing function
stop_words = set(stopwords.words('french'))

def preprocess_text(text):
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return tokens

# Apply FastText embeddings to each element in the Series
def get_text_embedding(text):
    preprocessed_text = preprocess_text(text)
    word_embeddings = [model.wv[word] for word in preprocessed_text if word in model.wv.key_to_index]
    
    if word_embeddings:
        text_embedding = np.mean(word_embeddings, axis=0)  # Use mean for aggregation
    else:
        text_embedding = np.zeros(model.vector_size)  # Use zero vector if no embeddings found
    
    return text_embedding


In [7]:
# Create a new column 'embedding' with the text embeddings
df_test['embedding_Commentaire'] = df_test['Commentaire'].apply(get_text_embedding)
df_test['embedding_Titres'] = df_test['Titres'].apply(get_text_embedding)

In [8]:
df_expanded = df_test['embedding_Commentaire'].apply(lambda x: pd.Series(x, dtype="float64")).add_prefix('vectComment_')
df_test = pd.concat([df_test, df_expanded], axis=1)
df_test = df_test.drop('embedding_Commentaire', axis=1)

df_expanded2 = df_test['embedding_Titres'].apply(lambda x: pd.Series(x, dtype="float64")).add_prefix('vectTitre_')
df_test = pd.concat([df_test, df_expanded2], axis=1)
df_test = df_test.drop('embedding_Titres', axis=1)


In [10]:
print(df_test.info())

#print(X_train.tolist()[0:5])
#print([x for x in X_train.apply(lambda x: len(x)) if x != 300])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285983 entries, 0 to 285982
Columns: 671 entries, index to vectTitre_299
dtypes: float64(602), int64(67), object(2)
memory usage: 1.4+ GB
None


In [26]:

df = df_test.filter(like = 'vect')
                    
df = pd.concat([df,df_test[['Nombre_avis_publie','longCommentaire','longTitres','nb_Mots_Commentaire','nb_Mots_Titres', 'nb_majuscules_Commentaire',
       'nb_chiffres_Commentaire', 'nb_ponctuation_Commentaire',
       'nb_special_Commentaire', 'nb_majuscules_Titre', 'nb_chiffres_Titre',
       'nb_ponctuation_Titre', 'nb_special_Titre', 'Sentiment']]],axis=1)
                                
                    
print(df.info())
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285983 entries, 0 to 285982
Columns: 614 entries, vectComment_0 to Sentiment
dtypes: float64(600), int64(14)
memory usage: 1.3 GB
None
   vectComment_0  vectComment_1  vectComment_2  vectComment_3  vectComment_4  \
0       0.011306       0.016411      -0.017874      -0.011596      -0.065694   
1       0.019220       0.009953      -0.026793      -0.008516      -0.085727   
2       0.014238       0.001802      -0.016503       0.002777      -0.093255   
3       0.008320      -0.009746      -0.042637      -0.001757      -0.087079   
4       0.030223       0.006146      -0.021092      -0.012093      -0.062279   
5       0.014521       0.008975      -0.038061      -0.007624      -0.109386   
6       0.021570       0.008288      -0.027958       0.000285      -0.086603   
7       0.016658      -0.001761      -0.030290      -0.014614      -0.080752   
8      -0.004167       0.025591      -0.054842      -0.022558      -0.037973   
9       0.00960

In [27]:
# (b) Division du DataFrame en ensembles d'entraînement et de test.

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns="Sentiment"), df['Sentiment'], test_size=0.2, random_state=42)

In [28]:
# (d) Création du classificateur RandomForest.
clf = make_pipeline(StandardScaler(with_mean=False), RandomForestClassifier(random_state=42))

In [29]:
# (e) Entraînement du classificateur sur l'ensemble d'entraînement.
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [30]:
# (f) Affichage du rapport de classification du modèle Gradient Boosting.
print("Rapport de classification :\n", classification_report(y_test, y_pred))

Rapport de classification :
               precision    recall  f1-score   support

           0       0.84      0.69      0.76     10645
           1       0.93      0.97      0.95     46552

    accuracy                           0.92     57197
   macro avg       0.89      0.83      0.85     57197
weighted avg       0.91      0.92      0.91     57197



In [31]:
# (g) Calcul et affichage de la matrice de confusion.
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Classe prédite,0,1
Classe réelle,,
0,7366,3279
1,1412,45140


# additionnal libraries

In [ ]:
from keras import Sequential
from keras.layers import Embedding, Dense, GlobalAveragePooling1D, Dropout

# Word Embedding 1: bag of word

In [ ]:
embedding_dim = 300
# Model
model = Sequential()
# 1. Embedding
model.add(Embedding(10000, embedding_dim))
# 2. Bag of Words
model.add(GlobalAveragePooling1D())
# 3. Classification
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit
history = model.fit(X_train, y_train,
    batch_size = 32,
    epochs = 5,
    validation_data = [X_test, y_test])

# Word Embedding 2: Vectorizer

In [ ]:
embedding_dim = 300
# Model
model = Sequential()
# 1. Embedding
model.add(Embedding(10000, embedding_dim))
# 2. Bag of Words
model.add(GlobalAveragePooling1D())
# 3. Classification
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit
history = model.fit(X_train, y_train,
    batch_size = 32,
    epochs = 5,
    validation_data = [X_test, y_test])